In [1]:
import torch
from dataclasses import dataclass

In [2]:
@dataclass
class gpuConfig:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_gpus: int = torch.cuda.device_count()
    device_name :str = torch.cuda.get_device_name()
    device_properties :str = torch.cuda.get_device_properties()

In [4]:
gpuConfig.device_name

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [ ]:
@dataclass
class dataConfig:
    data_path: str = "./data/brain_tumor"
    model_name: str = "nvidia/segformer-b0-finetuned-ade-512-512"
    batch_size: int = 32
    epochs: int = 5
    lr: float = 2e-5
    num_classes: int = 2 #background & Cell 

In [5]:
@dataclass
class DataPaths:
    images_root: str
    masks_root: str

In [6]:
from transformers import SegformerImageProcessor

In [7]:
def get_preprocessor(model_name):
    processor = SegformerImageProcessor.from_pretrained(model_name)
    
    def preprocess_pipeline(image, mask=None):
        if mask is not None:
            inputs = processor(images=image, segmentation_maps=mask, return_tensors="pt")
            return inputs["pixel_values"].squeeze(0), inputs["labels"].squeeze(0)
        else:
            inputs = processor(images=image, return_tensors="pt")
            return inputs["pixel_values"].squeeze(0), None

In [8]:
def get_dataloader(img_paths, mask_paths, preprocess_pipeline, batch_size, shuffle=True):
    dataset = SegmentationDataset(img_paths, mask_paths, transform=preprocess_pipeline)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)